# Create dataset

In [8]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = 'PYENx7ntOXOzvXfoRzICwWdDzb6wdBsl0E5FThtx'  # Get one at https://freesound.org/apiv2/apply/
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'duration', 'previews']
FILES_DIR = 'files'  # Will be relative to the current folder
N_SOUNDS_PER_QUERY = 5  # Max 150
DATASET_FILENAME = 'dataset.csv'

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [14]:
def query_freesound(query):
    pager = freesound_client.text_search(
        query = query,
        filter = 'duration:[0 TO 30]',  # Take sounds lasting less than 30 seconds
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = N_SOUNDS_PER_QUERY
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(fs_object):
    record = {key: fs_object.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in data frame
    record["path"] = "files/" + fs_object.previews.preview_hq_ogg.split("/")[-1]
    return record

def make_db(categories):
    sounds = sum([query_freesound(category) for category in categories],[])
    for count, sound in enumerate(sounds):
        print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
        retrieve_sound_preview(sound, 'files/')
    data_frame =  pd.DataFrame([make_pandas_record(s) for s in sounds])
    data_frame.to_csv(DATASET_FILENAME)
    print('Saved dataframe with basic sound matadata! {0}'.format(DATASET_FILENAME))
    return data_frame

animal_sounds = ["dog bark", "cat meow", "lion roar", "nightingale"]
df = make_db(animal_sounds)

Saved dataframe with basic sound matadata! dataset.csv


In [15]:
display(df)

,duration,id,name,path,username
0,21.758700,337101,Group_of_Dogs_Barking.WAV,files/337101_3474310-hq.ogg,ivolipa
1,6.472470,327666,Dog Bark.wav,files/327666_5632380-hq.ogg,Juan_Merie_Venter
2,8.867120,413758,dogbarking.mp3,files/413758_7958399-hq.ogg,lala_davis554
3,1.953880,163459,LBS_FX DOG Small Alert Bark001.wav,files/163459_2965892-hq.ogg,LittleBigSounds
4,29.000000,456943,Blossom Bark 29sec mix.wav,files/456943_3194431-hq.ogg,Zajjman
5,1.790840,415209,"Cat, Screaming, A.wav",files/415209_5121236-hq.ogg,InspectorJ
6,14.932700,213889,The cat's meow,files/213889_2287873-hq.ogg,Counter-gamer
7,6.225330,341545,Cat -meowing and scratching,files/341545_717697-hq.ogg,Nakhas
8,5.882330,365061,Pouring cat food into a plastic bowl,files/365061_6586255-hq.ogg,Ryntjie
9,1.544512,110011,cat meow,files/110011_1537422-hq.ogg,tuberatanka
